<a href="https://colab.research.google.com/github/Danddt/Russian_Opinion_Polls/blob/main/sumerize_art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [ ]:
# source code: https://www.activestate.com/blog/how-to-do-text-summarization-with-python/
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [ ]:
with open('/content/drive/MyDrive/news/Petro_Poroshenko.txt', 'r') as file:
    poroshenko = file.read().replace('\n', ' ')

In [ ]:
summarize(poroshenko, 0.05)

"We have enough weapons to resist, to stop Putin, but we definitely don't have enough weapons to push Putin out of Ukrainian territory, to launch a counteroffensive.And in this situation, time, combined with the number of weapons, are key factors.And in this situation, it becomes 10 times more difficult for the Ukrainian troops to destroy the Russian ones on Ukrainian territory."

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/news/bg_articles_clean_final.csv')
df.head()

,index,Date,Title,Content,link,Title_en,Content_en
0,0,2023-04-03,Задържаха 26-годишна за убийството на руски во...,Руската полиция задържа жена за убийството на...,https://nova.bg/news/view/2023/04/03/407128/%D...,They detained a 26-year-old woman for the murd...,Russian police have detained a woman for the m...
1,1,2023-04-01,Руският военен министър Сергей Шойгу обеща да ...,Русият военен министър Сергей Шойгу обеща да ...,https://nova.bg/news/view/2023/04/01/406878/%D...,Russian Defense Minister Sergei Shoigu promise...,Russian Defense Minister Sergei Shoigu has pro...
2,2,2023-04-01,Ескалация на дипломатическото напрежение между...,Дипломатическото напрежение между Москва и За...,https://nova.bg/news/view/2023/04/01/406869/%D...,Escalation of diplomatic tension between Mosco...,The diplomatic tension between Moscow and the ...
3,3,2023-03-30,Путин подписа указ за набиране на 147 000 войн...,Руският президент Владимир Путин подписа указ...,https://nova.bg/news/view/2023/03/30/406699/%D...,"Putin signed a decree to recruit 147,000 soldi...",Russian President Vladimir Putin has signed a ...
4,4,2023-03-30,Фон дер Лайен: Украйна е определящ фактор за в...,Председателят на ЕК Урсула фон дер Лайен пред...,https://nova.bg/news/view/2023/03/30/406652/%D...,Von der Leyen: Ukraine is a determining factor...,EC President Ursula von der Leyen warned China...


In [ ]:
df['Content_en'] =df['Content_en'].str.replace('NOVA news - now on Instagram, Twitter, Telegram and Viber - follow us.', '')

<ipython-input-85-5d96f1f57bb6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_en'] =df['Content_en'].str.replace('NOVA news - now on Instagram, Twitter, Telegram and Viber - follow us.', '')


In [ ]:
len_item = []
for item in df['Content_en']:
  item= len(str(item))
  len_item.append(item)

In [ ]:
df['len_art'] = len_item
df['len_art'].describe()

count      38.000000
mean     1573.473684
std      1000.255825
min       513.000000
25%       965.000000
50%      1344.500000
75%      1803.250000
max      5294.000000
Name: len_art, dtype: float64

In [ ]:
df[df['len_art']<1200].count()

index         15
Date          15
Title         15
Content       15
link          15
Title_en      15
Content_en    15
len_art       15
dtype: int64

In [ ]:
summery_list = []
for item in df['Content_en']:
  if len(str(item))>1300:
    temp = summarize(item, 0.1)
    summery_list.append(temp)
  if (len(str(item))<1300) and (len(str(item))>1000):
    temp = summarize(item, 0.2)
    summery_list.append(temp)
  if (len(str(item))<1000):
    temp = summarize(item, 0.5)
    summery_list.append(temp)

In [ ]:
len(summery_list)

38

In [ ]:
df['summerized'] =summery_list

In [ ]:
df.to_csv('/content/drive/MyDrive/news/article_and_summery.csv', index=False)